In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from cnn_class import cnn
import sklearn.cross_validation as cross_validation
from sklearn.cross_validation import cross_val_score, LeaveOneLabelOut
import time
import scipy.io as sio
from sklearn.metrics import classification_report, roc_auc_score, auc, roc_curve, f1_score
from RnnAttention.attention import attention
from scipy import interp
import os.path
from __future__ import  division, print_function, absolute_import
import tensorflow as tf

# read data
data = sio.loadmat('/home/yaoxiaojian/Desktop/kaggle/EEG/TACR_noRest_independent/T-set_sub6.mat')
print ('data reading ... done')
test_X	= data["test_x"] # [trials, channels, time length]
train_X	= data["train_x"]
test_y	= data["test_y"]
train_y = data["train_y"]
# train_X preprocess
X_inputs = np.transpose(train_X, [1, 0, 2])
X_inputs=X_inputs.reshape((X_inputs.shape[0],(X_inputs.shape[1]*X_inputs.shape[2])))
X_inputs = np.transpose(X_inputs, [1, 0])
X_inputs = X_inputs*100000
print ('X_inputs',X_inputs.shape)
# test_X preprocess
X_inputs1 = np.transpose(test_X, [1, 0, 2])
X_inputs1 = X_inputs1.reshape((X_inputs1.shape[0],(X_inputs1.shape[1]*X_inputs1.shape[2])))
X_inputs1 = np.transpose(X_inputs1, [1, 0])
X_inputs1 = X_inputs1*100000
print ('X_inputs1',X_inputs1.shape)

# X_inputs and Y_targets are np.array, while X,Y are tf.tensor class

learning_rate = 0.01
training_epochs = 10
batch_size = 751
display_step = 1
n_input = 32
X = tf.placeholder("float", [None, n_input])

n_hidden_1 = 88
n_hidden_2 = 44
n_hidden_3 = 22

weights = {
    'encoder_h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], )),
    'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], )),
    'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], )),
    'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_2], )),
    'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1], )),
    'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_1, n_input], )),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b3': tf.Variable(tf.random_normal([n_input])),
}


def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                     biases['encoder_b3'])

    return layer_3


def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    return layer_3

# iteration
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true = X

cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # cost func. MSE
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
encoder_result = []
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    total_batch = int(X_inputs.shape[0] / batch_size)
    for epoch in range(training_epochs):
        for i in range(total_batch):
            offset = (i * batch_size) 
            batch_xs = X_inputs[offset:(offset + batch_size), :] 
            #batch_ys = Y_targets[offset:(offset + batch_size), :]
            print ('.',end = '')
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        if epoch % display_step == 0:
            print()
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")
    
    # build encoder_result array as feature vector
    train_X=sess.run(encoder_op, feed_dict={X: X_inputs})      
    train_X=train_X.reshape(int(train_X.shape[0]/751),751,22)
    train_X=np.transpose(train_X,[0,2,1])
    
    test_X=sess.run(encoder_op, feed_dict={X: X_inputs1})      
    test_X=test_X.reshape(int(test_X.shape[0]/751),751,22)
    test_X=np.transpose(test_X,[0,2,1])

    print (train_X.shape, test_X.shape, train_y.shape,test_y.shape)
    display (test_X)

sess.close()

test_y	= test_y.ravel()
train_y = train_y.ravel()
display (test_y)

train_y = np.asarray(pd.get_dummies(train_y), dtype = np.int8)
test_y = np.asarray(pd.get_dummies(test_y), dtype = np.int8)

#############
# Set window
#############

window_size = 100
step = 50
n_channel = 22

def windows(data, size, step):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += step

def segment_signal_without_transition(data, window_size, step):
    segments = []
    for (start, end) in windows(data, window_size, step):
        if(len(data[start:end]) == window_size):
            segments = segments + [data[start:end]]
    return np.array(segments)

def segment_dataset(X, window_size, step):
    win_x = []
    for i in range(X.shape[0]):
        win_x = win_x + [segment_signal_without_transition(X[i], window_size, step)]
    win_x = np.array(win_x)
    return win_x

train_raw_x = np.transpose(train_X, [0, 2, 1])
test_raw_x = np.transpose(test_X, [0, 2, 1])


train_win_x = segment_dataset(train_raw_x, window_size, step)
print("train_win_x shape: ", train_win_x.shape)
test_win_x = segment_dataset(test_raw_x, window_size, step)
print("test_win_x shape: ", test_win_x.shape)

# [trial, window, channel, time_length]
train_win_x = np.transpose(train_win_x, [0, 1, 3, 2])
print("train_win_x shape: ", train_win_x.shape)

test_win_x = np.transpose(test_win_x, [0, 1, 3, 2])
print("test_win_x shape: ", test_win_x.shape)


# [trial, window, channel, time_length, 1]
train_x = np.expand_dims(train_win_x, axis = 4)
test_x = np.expand_dims(test_win_x, axis = 4)

num_timestep = train_x.shape[1]
###########################################################################
# set model parameters
###########################################################################
# kernel parameter
kernel_height_1st	= 22
kernel_width_1st 	= 45

kernel_stride		= 1

conv_channel_num	= 40

# pooling parameter
pooling_height_1st 	= 1
pooling_width_1st 	= 56

pooling_stride_1st = 10

# full connected parameter
attention_size = 512
n_hidden_state = 64

###########################################################################
# set dataset parameters
###########################################################################
# input channel
input_channel_num = 1

# input height 
input_height = train_x.shape[2]

# input width
input_width = train_x.shape[3]

# prediction class
num_labels = 2
###########################################################################
# set training parameters
###########################################################################
# set learning rate
learning_rate = 1e-4

# set maximum traing epochs
training_epochs = 100

# set batch size
batch_size = 10

# set dropout probability
dropout_prob = 0.5

# set train batch number per epoch
batch_num_per_epoch = train_x.shape[0]//batch_size

# instance cnn class
padding = 'VALID'

cnn_2d = cnn(padding=padding)

# input placeholder
X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, input_channel_num], name = 'X')
Y = tf.placeholder(tf.float32, shape=[None, num_labels], name = 'Y')
train_phase = tf.placeholder(tf.bool, name = 'train_phase')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# first CNN layer
conv_1 = cnn_2d.apply_conv2d(X, kernel_height_1st, kernel_width_1st, input_channel_num, conv_channel_num, kernel_stride, train_phase)
print("conv 1 shape: ", conv_1.get_shape().as_list())
pool_1 = cnn_2d.apply_max_pooling(conv_1, pooling_height_1st, pooling_width_1st, pooling_stride_1st)
print("pool 1 shape: ", pool_1.get_shape().as_list())

pool1_shape = pool_1.get_shape().as_list()
pool1_flat = tf.reshape(pool_1, [-1, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

fc_drop = tf.nn.dropout(pool1_flat, keep_prob)	

lstm_in = tf.reshape(fc_drop, [-1, num_timestep, pool1_shape[1]*pool1_shape[2]*pool1_shape[3]])

################
# pipline design
#################

########################## RNN ########################
cells = []
for _ in range(2):
	cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_state, forget_bias=1.0, state_is_tuple=True)
	cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
	cells.append(cell)
lstm_cell = tf.contrib.rnn.MultiRNNCell(cells)

init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)

# output ==> [batch, step, n_hidden_state]
rnn_op, states = tf.nn.dynamic_rnn(lstm_cell, lstm_in, initial_state=init_state, time_major=False)

########################## attention ########################
with tf.name_scope('Attention_layer'):
    attention_op, alphas = attention(rnn_op, attention_size, time_major = False, return_alphas=True)

attention_drop = tf.nn.dropout(attention_op, keep_prob)	

########################## readout ########################
y_ = cnn_2d.apply_readout(attention_drop, rnn_op.shape[2].value, num_labels)

# probability prediction 
y_prob = tf.nn.softmax(y_, name = "y_prob")

# class prediction 
y_pred = tf.argmax(y_prob, 1, name = "y_pred")

########################## loss and optimizer ########################
# cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y), name = 'loss')


update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
# set training SGD optimizer
	optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# get correctly predicted object
correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(Y, 1))

########################## define accuracy ########################
# calculate prediction accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

#############
# train test 
#############
def multiclass_roc_auc_score(y_true, y_score):
    assert y_true.shape == y_score.shape
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = y_true.shape[1]
    # compute ROC curve and ROC area for each class
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # compute macro-average ROC curve and ROC area
    # First aggregate all false probtive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    # Finally average it and compute AUC
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    return roc_auc
# run with gpu memory growth
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

train_acc = []
test_acc = []
best_test_acc = []
train_loss = []
with tf.Session(config=config) as session:
	session.run(tf.global_variables_initializer())
	best_acc = 0
	for epoch in range(training_epochs):
		pred_test = np.array([])
		true_test = []
		prob_test = []
		########################## training process ########################
		for b in range(batch_num_per_epoch):
			offset = (b * batch_size) % (train_y.shape[0] - batch_size) 
			batch_x = train_x[offset:(offset + batch_size), :, :, :, :]
			batch_x = batch_x.reshape([len(batch_x)*num_timestep, n_channel, window_size, 1])
			batch_y = train_y[offset:(offset + batch_size), :]
			_, c = session.run([optimizer, cost], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1-dropout_prob, train_phase: True})
		# calculate train and test accuracy after each training epoch
		if(epoch%1 == 0):
			train_accuracy 	= np.zeros(shape=[0], dtype=float)
			test_accuracy	= np.zeros(shape=[0], dtype=float)
			train_l 		= np.zeros(shape=[0], dtype=float)
			test_l			= np.zeros(shape=[0], dtype=float)
			# calculate train accuracy after each training epoch
			for i in range(batch_num_per_epoch):
				########################## prepare training data ########################
				offset = (i * batch_size) % (train_y.shape[0] - batch_size) 
				train_batch_x = train_x[offset:(offset + batch_size), :, :, :]
				train_batch_x = train_batch_x.reshape([len(train_batch_x)*num_timestep, n_channel, window_size, 1])
				train_batch_y = train_y[offset:(offset + batch_size), :]

				########################## calculate training results ########################
				train_a, train_c = session.run([accuracy, cost], feed_dict={X: train_batch_x, Y: train_batch_y, keep_prob: 1.0, train_phase: False})
				
				train_l = np.append(train_l, train_c)
				train_accuracy = np.append(train_accuracy, train_a)
			print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, " Training Cost: ", np.mean(train_l), "Training Accuracy: ", np.mean(train_accuracy))
			train_acc = train_acc + [np.mean(train_accuracy)]
			train_loss = train_loss + [np.mean(train_l)]
			# calculate test accuracy after each training epoch
			for j in range(batch_num_per_epoch):
				########################## prepare test data ########################
				offset = (j * batch_size) % (test_y.shape[0] - batch_size) 
				test_batch_x = test_x[offset:(offset + batch_size), :, :, :]
				test_batch_x = test_batch_x.reshape([len(test_batch_x)*num_timestep, n_channel, window_size, 1])
				test_batch_y = test_y[offset:(offset + batch_size), :]
				
				########################## calculate test results ########################
				test_a, test_c, prob_v, pred_v = session.run([accuracy, cost, y_prob, y_pred], feed_dict={X: test_batch_x, Y: test_batch_y, keep_prob: 1.0, train_phase: False})
				
				test_accuracy = np.append(test_accuracy, test_a)
				test_l = np.append(test_l, test_c)
				pred_test = np.append(pred_test, pred_v)
				true_test.append(test_batch_y)
				prob_test.append(prob_v)
			if np.mean(test_accuracy) > best_acc :
				best_acc = np.mean(test_accuracy)
			true_test = np.array(true_test).reshape([-1, num_labels])
			prob_test = np.array(prob_test).reshape([-1, num_labels])
			auc_roc_test = multiclass_roc_auc_score(y_true=true_test, y_score=prob_test)
			f1 = f1_score (y_true=np.argmax(true_test, axis = 1), y_pred=pred_test, average = 'macro')
			print("("+time.asctime(time.localtime(time.time()))+") Epoch: ", epoch+1, "Test Cost: ", np.mean(test_l), 
																					  "Test Accuracy: ", np.mean(test_accuracy), 
																					  "Test f1: ", f1, 
																					  "Test AUC: ", auc_roc_test['macro'], "\n")


/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


data reading ... done
X_inputs (4295720, 32)
X_inputs1 (390520, 32)
Instructions for updating:
Colocations handled automatically by placer.
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

...

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

array([[[-9.1499579e-01,  5.1997077e-01, -2.7735600e+00, ...,
         -4.9203885e-01, -5.2881441e+00, -5.5709314e+00],
        [ 4.7081161e+00,  2.9546387e+00,  2.7997313e+00, ...,
         -1.0539021e+00, -2.2881699e+00, -2.7551341e+00],
        [-3.4530756e+00,  3.2621735e-01,  8.5119015e-01, ...,
         -1.7234206e-02,  2.7477996e+00,  2.8135498e+00],
        ...,
        [-4.3167267e+00, -9.1761851e-01, -8.1773925e-01, ...,
         -2.0775449e+00, -2.2141092e+00, -2.2069318e+00],
        [-7.9001307e-02,  1.2160742e+00, -3.0100241e+00, ...,
          3.6796041e+00,  5.0075903e+00,  5.2259855e+00],
        [ 7.0494990e+00,  1.8895900e+00,  2.6254556e+00, ...,
         -3.1519630e+00, -1.4910467e+00, -1.3786123e+00]],

       [[-3.9436066e+00, -2.0009608e+00, -8.6219609e-01, ...,
         -2.6144137e+00, -2.0352993e+00, -1.9728140e+00],
        [-3.8296909e+00, -4.9383025e+00, -3.6101451e+00, ...,
          5.0646420e+00,  4.4122319e+00,  4.5899506e+00],
        [ 7.9211636e+00, 

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

train_win_x shape:  (5720, 14, 100, 22)
test_win_x shape:  (520, 14, 100, 22)
train_win_x shape:  (5720, 14, 22, 100)
test_win_x shape:  (520, 14, 22, 100)
Instructions for updating:
Use keras.layers.batch_normalization instead.
conv 1 shape:  [None, 1, 56, 40]
pool 1 shape:  [None, 1, 1, 40]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to thi

(Fri Dec  4 16:46:25 2020) Epoch:  21 Test Cost:  0.49116839166287773 Test Accuracy:  0.7713286697968736 Test f1:  0.7711876567286382 Test AUC:  0.8625299423868312 

(Fri Dec  4 16:46:59 2020) Epoch:  22  Training Cost:  0.4585918571305025 Training Accuracy:  0.792832166112803
(Fri Dec  4 16:47:03 2020) Epoch:  22 Test Cost:  0.5104767659327367 Test Accuracy:  0.750349647098488 Test f1:  0.7472246002835188 Test AUC:  0.8607807815529029 

(Fri Dec  4 16:47:36 2020) Epoch:  23  Training Cost:  0.46605027961981044 Training Accuracy:  0.7867132856191456
(Fri Dec  4 16:47:40 2020) Epoch:  23 Test Cost:  0.4854155174502126 Test Accuracy:  0.7748251730849692 Test f1:  0.7745212598280606 Test AUC:  0.8657346111621264 

(Fri Dec  4 16:48:16 2020) Epoch:  24  Training Cost:  0.4472213752463564 Training Accuracy:  0.8010489485272161
(Fri Dec  4 16:48:19 2020) Epoch:  24 Test Cost:  0.5087795211718633 Test Accuracy:  0.7459790179362664 Test f1:  0.7418082228412782 Test AUC:  0.8680102098878799 

(